1\. Write a function that converts number representation (bin<->dec<->hex)

In [ ]:
s=''

s='10'
a=int(s)
print(s[1])

In [ ]:
#se numero intero, base_1 è la base di partenza mentre base_2 è quella finale
def conversion(number, base_1, base_2):
    
    if base_1>9 and base_1<11: #caso in cui la base di partenza è decimale, il numero è int
        cn_str=''
        converted_number=0
        while number>0: 
            converted_number = number%base_2
            cn_str += str(converted_number)
            number=number/base_2
        cn_str=cn_str[::-1]
        return cn_str
    else: #se la base di partenza non è decimale il numero è una stringa
        converted_number=0
        n_str=str(number)
        n_str=n_str[::-1]
        l=len(n_str)
        for i in range(l):
            converted_number +=int(n_str[i])*base_1**i
            
        return converted_number

print(conversion(1000,2,10))

##bisognerebbe gestire anche parte frazionaria e i casi d'errore!!!!!!
        

2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [ ]:
s='0.110'
print(float(s,2))

In [ ]:
def spfp_conversion(word):
    print(word)
    word=word[::-1]
    print(len(word))
    segno=word[31]
    e=int(word[24:31],2)
    bias=127
    esp=e-bias
    mantissa=int(word[0:24],2)
    number = (1+ mantissa*0.1)*2**esp
    if segno=='0':
        return '+'+str(number)
    else:
        return '-'+str(number)

spfp_conversion('10011111100000000000000000000001')

3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [3]:
detect_under=float(1)
detect_over=float(1)
i=0
m=0
while detect_under>0:
    i +=1
    detect_under=detect_under/2
print(detect_under)
print(i)

while detect_over<float('Inf'):
    m+=1
    detect_over=detect_over*2
    
print(detect_over)
print(m)
print(2**m)


0.0
1075
inf
1024
179769313486231590772930519078902473361797697894230657273430081157732675805500963132708477322407536021120113879871393357658789768814416622492847430639474124377767893424865485276302219601246094119453082952085005768838150682342462881473913110540827237163350510684586298239947245938479716304835356329624224137216


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [6]:
#It is the difference between 1 and the next nearest number representable as a machine number. 
# quindi è l'errore massimo che faccio quando memorizzo numeri
var=float(1)
eps=float(1)
it=0
while (var + eps) > var:
    eps=eps/2
    it +=1
print(eps)
print(it)
print(var+eps)

1.11022302463e-16
53
1.0


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [ ]:
import math
math.sqrt(4)
print(3/4)

In [9]:
import math
def sol_eq(a,b,c):
    d=math.sqrt(b**2-4*a*c)
    b1=~b+1
    x1=(b1+d)/(2*a)
    x2=(b1-d)/(2*a)
    return x1, x2

print(sol_eq(2,3,1))
print(sol_eq(0.001,1000,0.001))

def sol_eq_2(a,b,c):
    d=math.sqrt(b**2-4*a*c)
    b1=~b+1
    x1=((b1+d)*(b1-d))/((2*a)*(b1-d))
    x2=((b1-d)*(b1+d))/((2*a)*(b1+d))
    return x1, x2

print(sol_eq_2(0.001,1000,0.001))

def sol_eq_3(a,b,c):
    a=round(a)
    
    c=round(c)
    
    b1=~b+1
    b1=round(b1)
    b=round(b)
    d=math.sqrt(b**2-4*a*c)
    x1=(b1+d)/(2*a)
    x2=(b1-d)/(2*a)
    return x1, x2

print(sol_eq(2,3,1))
print(sol_eq_3(0.001,1000,0.001))

(-0.5, -1.0)
(-9.999894245993346e-07, -999999.999999)
(-9.999894245993346e-07, -999999.9999990001)
(-0.5, -1.0)


ZeroDivisionError: float division by zero

6\. Write a program that implements a function $f(x)$ returning the value of $x(x−1)$

(a)calculates the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with δ = 10−2. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

7\. Calculate the integral